In [1]:
import pandas as pd
import numpy as np

In [75]:
# 인구데이터 처리
df_pop = pd.read_excel("data/population_full.xlsx")
df_pop['발생장소_동'] = df_pop['발생장소_동'].apply(lambda x: x.lstrip())
df_pop['행정동명'] = df_pop['발생장소_시'] + " " + df_pop['발생장소_구'] + " " + df_pop['발생장소_동']
df_pop = df_pop.loc[:, ['행정동명'] + [i for i in range(2014, 2024)]]
df_pop.head(3)

,행정동명,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,충청남도 천안시 동남구 목천읍,27449,27281,26871,26475,25174,24659,24224,24274,24122,23542
1,충청남도 천안시 동남구 풍세면,4689,4599,4481,4390,4312,4255,4165,4107,4080,3981
2,충청남도 천안시 동남구 광덕면,4459,4456,4561,4530,4392,4365,4344,4295,4227,4173


In [76]:
df_code = pd.read_excel("data/code_var.xlsx")
df_code = df_code[df_code['시도'] == '충청남도']
df_code = df_code.loc[df_code['행정구역코드'].fillna(3333333).apply(lambda x: len(str(int(x))) > 6)].reset_index(drop=True)
df_code['법정동코드'] = df_code['법정동코드'].apply(lambda x: str(x)[:-2])
df_code['행정동명'] = df_code['시도'] + " " + df_code['시군구'] + " " + df_code['행정구역명']
df_code = df_code.loc[:, ['행정동명', '법정동코드']]
df_code = df_code.groupby(['행정동명', '법정동코드']).count().reset_index()
# 오천면어항출장소는 신흑동에 포함되는 것으로 처리
df_code.loc[df_code['행정동명'] == '충청남도 보령시 오천면어항출장소', '법정동코드'] = '44180110'
#df_code.to_excel('code1.xlsx')

In [77]:
# 법정동코드 불러오기
df_law = pd.read_table("data/법정동코드 전체자료.txt", sep='\t', encoding='cp949')
# 읍면동 단위로 정제
chgnam_code = df_law[df_law['법정동코드'].apply(lambda x: str(x).startswith('44'))]
chgnam_code = chgnam_code[chgnam_code['폐지여부'] == '존재']
chgnam_code = chgnam_code[chgnam_code['법정동명'].apply(lambda x: x[-1] in ('읍', '면', '동'))]
chgnam_code.loc[:, '법정동코드'] = chgnam_code.loc[:, '법정동코드'].apply(lambda x: str(x)[:-2])
chgnam_code.reset_index(drop=True, inplace=True)
chgnam_code.drop(columns='폐지여부', inplace=True)
print(len(chgnam_code))
chgnam_code.head(3)

285


,법정동코드,법정동명
0,44131101,충청남도 천안시 동남구 대흥동
1,44131102,충청남도 천안시 동남구 성황동
2,44131103,충청남도 천안시 동남구 문화동


In [106]:
df_x = pd.merge(df_code, chgnam_code, how='left', on='법정동코드')
df_x.head(3)

,행정동명,법정동코드,법정동명
0,충청남도 계룡시 금암동,44250101,충청남도 계룡시 금암동
1,충청남도 계룡시 두마면,44250310,충청남도 계룡시 두마면
2,충청남도 계룡시 신도안면,44250330,충청남도 계룡시 신도안면


In [145]:
df_pop_law = pd.merge(df_pop, df_x, how='left', on='행정동명')
df_pop_law.head(3)

,행정동명,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,법정동코드,법정동명
0,충청남도 천안시 동남구 목천읍,27449,27281,26871,26475,25174,24659,24224,24274,24122,23542,44131250,충청남도 천안시 동남구 목천읍
1,충청남도 천안시 동남구 풍세면,4689,4599,4481,4390,4312,4255,4165,4107,4080,3981,44131310,충청남도 천안시 동남구 풍세면
2,충청남도 천안시 동남구 광덕면,4459,4456,4561,4530,4392,4365,4344,4295,4227,4173,44131320,충청남도 천안시 동남구 광덕면


In [142]:
dup = pd.DataFrame(df_pop_law['행정동명'].value_counts()).reset_index()
dup.head(3)

,행정동명,count
0,충청남도 서산시 석남동,7
1,충청남도 논산시 취암동,7
2,충청남도 공주시 월송동,7


In [146]:
df_pop_law['중복개수'] = df_pop_law['행정동명'].apply(lambda x: dup.loc[dup['행정동명'] == x, 'count'].values[0])
for i in range(2014, 2024):
    df_pop_law[i] = df_pop_law[i] / df_pop_law['중복개수']
df_pop_law.head(3)

,행정동명,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,법정동코드,법정동명,중복개수
0,충청남도 천안시 동남구 목천읍,27449.0,27281.0,26871.0,26475.0,25174.0,24659.0,24224.0,24274.0,24122.0,23542.0,44131250,충청남도 천안시 동남구 목천읍,1
1,충청남도 천안시 동남구 풍세면,4689.0,4599.0,4481.0,4390.0,4312.0,4255.0,4165.0,4107.0,4080.0,3981.0,44131310,충청남도 천안시 동남구 풍세면,1
2,충청남도 천안시 동남구 광덕면,4459.0,4456.0,4561.0,4530.0,4392.0,4365.0,4344.0,4295.0,4227.0,4173.0,44131320,충청남도 천안시 동남구 광덕면,1


In [147]:
df_pop_law = df_pop_law.drop(columns=['행정동명', '중복개수'])

In [161]:
df_pop_law = pd.DataFrame(df_pop_law.groupby('법정동코드').sum()).reset_index()
df_pop_law = df_pop_law.loc[:, ['법정동명', '법정동코드'] + [i for i in range(2014, 2024)]]
df_pop_law.head(3)

,법정동명,법정동코드,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,충청남도 천안시 동남구 대흥동,44131101,1498.500000,1486.750000,1464.500000,1409.500000,1426.25,1409.000000,1390.750000,1346.500000,1349.750000,1300.500000
1,충청남도 천안시 동남구 성황동,44131102,1681.000000,1602.333333,1543.000000,1479.333333,1387.00,1305.000000,1226.000000,1435.333333,2214.666667,2628.333333
2,충청남도 천안시 동남구 문화동충청남도 천안시 동남구 문화동,44131103,5186.666667,5041.000000,4822.333333,4631.000000,4472.00,4299.666667,4184.666667,4317.000000,5059.333333,5376.666667
